In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from torch import nn
import torch.nn.functional as F

from torch.autograd import Variable
import torch.utils.data as Data
import torchvision

from torchvision import transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.patheffects as PathEffects
# from centerLoss import CenterLoss
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.cm as cm
%matplotlib inline

In [2]:
print(f"torch.cuda.is_available() = {torch.cuda.is_available()}")
print(f"torch.cuda.device_count() = {torch.cuda.device_count()}")
if 'CUDA_VISIBLE_DEVICES' in os.environ:
    print(f"os.environ['CUDA_VISIBLE_DEVICES'] = {os.environ['CUDA_VISIBLE_DEVICES']}")
else:
    print(f"没有设置环境变量 'CUDA_VISIBLE_DEVICES'")

torch.cuda.is_available() = True
torch.cuda.device_count() = 1
os.environ['CUDA_VISIBLE_DEVICES'] = 3


In [3]:
data_root = {
    "train":"train",
    "valid":"valid",
    "test":"test"
}

In [4]:
def train_tf(x):
   
    im_aug = transforms.Compose([
        transforms.Resize(120),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(96),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    x = im_aug(x)
    return x

def test_tf(x):
   
    im_aug = transforms.Compose([
        transforms.Resize(120),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    x = im_aug(x)
    return x

In [5]:
init_seed = 1
torch.manual_seed(init_seed)
torch.cuda.manual_seed(init_seed)
np.random.seed(init_seed) 

In [6]:
X1=ImageFolder(data_root["train"],transform=train_tf)  ## lazy loding
X2=ImageFolder(data_root["valid"],transform=train_tf)  ## lazy loding

X=torch.utils.data.ConcatDataset([X1,X2])
X_train,X_valid = torch.utils.data.random_split(X, [160000, 20000])
train_dataloader= DataLoader(X_train,batch_size=64,shuffle=True)
valid_dataloader= DataLoader(X_valid,batch_size=64,shuffle=True)

In [7]:
# X_train=ImageFolder(data_root["train"],transform=train_tf)  ## lazy loding
# train_dataloader= DataLoader(X_train,batch_size=64,shuffle=True)

In [8]:
# X_valid=ImageFolder(data_root["valid"],transform=test_tf)  ## lazy loding
# valid_dataloader= DataLoader(X_valid,batch_size=64,shuffle=True)

In [9]:
class CrossEntropyLabelSmooth(nn.Module):

    def __init__(self, num_classes, epsilon):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros_like(log_probs).scatter_(1, targets.unsqueeze(1), 1)
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (-targets * log_probs).mean(0).sum()
        return loss
    
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.extract = torch.nn.Sequential(
            torchvision.models.resnet152(pretrained=True).conv1,
            torchvision.models.resnet152(pretrained=True).bn1,
            torchvision.models.resnet152(pretrained=True).relu,
            torchvision.models.resnet152(pretrained=True).maxpool,
            torchvision.models.resnet152(pretrained=True).layer1,
            torchvision.models.resnet152(pretrained=True).layer2,
            torchvision.models.resnet152(pretrained=True).layer3,
            torchvision.models.resnet152(pretrained=True).layer4,
            torchvision.models.resnet152(pretrained=True).avgpool,   
        )
        
        self.feat = torch.nn.Sequential(
            torch.nn.Linear(2048, 256),
        )

        for index,net in enumerate(self.feat):
            if index==0 :
                torch.nn.init.xavier_uniform_(net.weight)
                
        self.pred = torch.nn.Sequential(
            torch.nn.Linear(256, 10),
        )
        
        for index,net in enumerate(self.pred):
            if index==0:
                torch.nn.init.xavier_uniform_(net.weight)
                
    def forward(self, x):
        x = self.extract(x)    
        x = x.view(-1, 2048)
        feat = self.feat(x)
        pred = F.log_softmax(self.pred(feat),1)

        return feat ,pred

In [10]:
model = ConvNet()
# checkpoint = torch.load("old_model/init1227.pkl")
checkpoint = torch.load("old_model/finetune1227.pkl")
model.load_state_dict(checkpoint['model'])
# optimizer4nn =torch.optim.Adam(model.parameters(),lr=1e-3)

for param in model.parameters():
    param.requires_grad = False
for param in model.feat.parameters():
    param.requires_grad = True
for param in model.pred.parameters():
    param.requires_grad = True
    
model=model.cuda()
optimizer4nn = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), 
                               lr=1e-3, 
                               momentum=0.9, 
                               weight_decay=0.0005)    

nllloss = CrossEntropyLabelSmooth(10,0.0).cuda()
# scheduler = lr_scheduler.StepLR(optimizer4nn, 20, gamma=0.8)
scheduler  = lr_scheduler.CosineAnnealingLR(optimizer4nn, T_max=5,eta_min=4e-08)
# nllloss = torch.nn.CrossEntropyLoss().cuda()

In [11]:
# startepoch=checkpoint["epoch"]+1
# acc=checkpoint["valid_acc"]
# acc2=checkpoint["train_acc"]
# startepoch,acc,acc2

In [12]:
from tqdm import tqdm_notebook
def train(train_loader, model, epoch):
    print("Training Epoch: {}".format(epoch))
    model.train()

    train_precision=0.0
    for step, (data, target) in enumerate(train_loader,1):
        data = Variable(data).cuda()
        target = Variable(target).cuda()
        
        feat, pred = model(data)
        _, pred_label = pred.max(dim=1)
        loss=nllloss(pred, target)
        
        optimizer4nn.zero_grad()
        loss.backward()
        optimizer4nn.step()

        train_precision += float(torch.sum(pred_label== target)) 
        if step % 350 == 0:
            print("Epoch: {} step: {} loss:{}".format(epoch, step,loss))
            
#     train_precision /= 90000.0
    train_precision /= 160000.0
    
    print("train accuracy: {}%".format(train_precision * 100))
    return train_precision
            
def test(test_loader, model, epoch):
    print("Predicting Epoch: {}".format(epoch))
    model.eval()
    
#     total_target = []
#     total_feature = []
    precision=0.0
    for (data, target) in tqdm_notebook(test_loader):
        data = Variable(data).cuda()
        target = Variable(target).cuda()

        feature, pred = model(data)
        _, pred_label = pred.max(dim=1)
        
        precision += float(torch.sum(pred_label == target))

#     precision /=90000.0
    precision /=20000.0
    print("Validation accuracy: {}%".format(precision * 100))

    return precision

    
startepoch=0
best_precision=0.88
for epoch in range(startepoch,startepoch+40):
    train_precision=train(train_dataloader, model, epoch)
    precision=test(valid_dataloader, model, epoch)
    
    scheduler.step()
    if precision>best_precision:
        best_precision=precision
        state = {"model":model.state_dict(),"epoch":epoch,
                 "valid_acc":precision,
                 "train_acc":train_precision,
            }
        PATH ="old_model/finetune1230.pkl"
        torch.save(state, PATH)

Training Epoch: 0
Epoch: 0 step: 350 loss:0.21694651246070862
Epoch: 0 step: 700 loss:0.43420904874801636
Epoch: 0 step: 1050 loss:0.6255621910095215
Epoch: 0 step: 1400 loss:0.32034799456596375
Epoch: 0 step: 1750 loss:0.39717164635658264
Epoch: 0 step: 2100 loss:0.3900606632232666
Epoch: 0 step: 2450 loss:0.40450114011764526
train accuracy: 90.288125%
Predicting Epoch: 0



Validation accuracy: 90.765%
Training Epoch: 1
Epoch: 1 step: 350 loss:0.17803439497947693
Epoch: 1 step: 700 loss:0.2242719531059265
Epoch: 1 step: 1050 loss:0.41016799211502075
Epoch: 1 step: 1400 loss:0.19362665712833405
Epoch: 1 step: 1750 loss:0.35945403575897217
Epoch: 1 step: 2100 loss:0.2527996301651001
Epoch: 1 step: 2450 loss:0.24546442925930023
train accuracy: 90.35625%
Predicting Epoch: 1



Validation accuracy: 90.66499999999999%
Training Epoch: 2
Epoch: 2 step: 350 loss:0.46262383460998535
Epoch: 2 step: 700 loss:0.569359540939331
Epoch: 2 step: 1050 loss:0.31949594616889954
Epoch: 2 step: 1400 loss:0.1229654848575592
Epoch: 2 step: 1750 loss:0.2942013442516327
Epoch: 2 step: 2100 loss:0.2693118155002594
Epoch: 2 step: 2450 loss:0.519070029258728
train accuracy: 90.451875%
Predicting Epoch: 2



Validation accuracy: 90.85%
Training Epoch: 3
Epoch: 3 step: 350 loss:0.09342627227306366
Epoch: 3 step: 700 loss:0.3358127474784851
Epoch: 3 step: 1050 loss:0.2104664444923401
Epoch: 3 step: 1400 loss:0.11204127222299576
Epoch: 3 step: 1750 loss:0.369768351316452
Epoch: 3 step: 2100 loss:0.31629154086112976
Epoch: 3 step: 2450 loss:0.436606764793396
train accuracy: 90.51625%
Predicting Epoch: 3



Validation accuracy: 91.05%
Training Epoch: 4
Epoch: 4 step: 350 loss:0.14059333503246307
Epoch: 4 step: 700 loss:0.19515332579612732
Epoch: 4 step: 1050 loss:0.21391482651233673
Epoch: 4 step: 1400 loss:0.39010530710220337
Epoch: 4 step: 1750 loss:0.38953685760498047
Epoch: 4 step: 2100 loss:0.20347608625888824
Epoch: 4 step: 2450 loss:0.33661043643951416
train accuracy: 90.47812499999999%
Predicting Epoch: 4


...


Validation accuracy: 90.84%
Training Epoch: 36
Epoch: 36 step: 350 loss:0.23080262541770935
Epoch: 36 step: 700 loss:0.3409324288368225
Epoch: 36 step: 1050 loss:0.39427757263183594
Epoch: 36 step: 1400 loss:0.22992490231990814
Epoch: 36 step: 1750 loss:0.5751850605010986
Epoch: 36 step: 2100 loss:0.10465699434280396
Epoch: 36 step: 2450 loss:0.24853749573230743
train accuracy: 90.71375%
Predicting Epoch: 36



Validation accuracy: 91.065%
Training Epoch: 37
Epoch: 37 step: 350 loss:0.4098493754863739
Epoch: 37 step: 700 loss:0.15978536009788513
Epoch: 37 step: 1050 loss:0.24787214398384094
Epoch: 37 step: 1400 loss:0.2622033357620239
Epoch: 37 step: 1750 loss:0.34157612919807434
Epoch: 37 step: 2100 loss:0.28456681966781616
Epoch: 37 step: 2450 loss:0.25412946939468384
train accuracy: 90.67875%
Predicting Epoch: 37



Validation accuracy: 90.875%
Training Epoch: 38
Epoch: 38 step: 350 loss:0.30008429288864136
Epoch: 38 step: 700 loss:0.4284592866897583
Epoch: 38 step: 1050 loss:0.3955312967300415
Epoch: 38 step: 1400 loss:0.3480586111545563
Epoch: 38 step: 1750 loss:0.25641489028930664
Epoch: 38 step: 2100 loss:0.39260774850845337
Epoch: 38 step: 2450 loss:0.2641873061656952
train accuracy: 90.60249999999999%
Predicting Epoch: 38



Validation accuracy: 91.11%
Training Epoch: 39
Epoch: 39 step: 350 loss:0.28312966227531433
Epoch: 39 step: 700 loss:0.40904000401496887
Epoch: 39 step: 1050 loss:0.21202945709228516
Epoch: 39 step: 1400 loss:0.3494466543197632
Epoch: 39 step: 1750 loss:0.20746847987174988
Epoch: 39 step: 2100 loss:0.4464390277862549
Epoch: 39 step: 2450 loss:0.699643611907959
train accuracy: 90.631875%
Predicting Epoch: 39



Validation accuracy: 90.96499999999999%


In [13]:
exit()

In [ ]:
177.77
22.22

In [12]:
# def vis(feat, labels, epoch):
#     sns.set_style("white")
#     visual_pd=pd.DataFrame(feat,columns=["0","1","2"])
#     visual_pd["labels"]= labels
    
#     sns.pairplot(visual_pd,
#                 kind = 'scatter',  # 散点图/回归分布图 {‘scatter’, ‘reg’}  
#                 diag_kind="hist",  # 直方图/密度图 {‘hist’, ‘kde’}
#                 hue="labels",     # 按照某一字段进行分类
#                 size = 3,   # 图表大小
#                 )
#     plt.savefig('./images/epoch=%d.jpg' % epoch)
    
# target=np.random.randint(1,10,size=(90000,))
# vis(feature_draw, target, epoch)

In [12]:
X_test=ImageFolder(data_root["test"],transform=test_tf)  ## lazy loding
test_dataloader= DataLoader(X_test,batch_size=64,shuffle=True)

In [13]:
Y_predlist=[]
Y_test=[]
from tqdm import tqdm_notebook
for (X,Y) in tqdm_notebook(test_dataloader):
    model.eval()
    X,Y=X.cuda(),Y.cuda()
    model=model.cuda()
    _,Y_pred=model(X)
#     print(Y_pred)
    Y_pred1=torch.argmax(Y_pred,axis=1)
    Y_predlist.extend(Y_pred1)
    Y_test.extend(Y)
Y_test=torch.Tensor(Y_test)
Y_predlist=torch.Tensor(Y_predlist)

In [14]:
acc=float(torch.sum(Y_predlist==Y_test))/float(len(Y_test))
acc

0.8935444444444445